# I/O of molecular dynamics trajectory

In [1]:
!fishmol

                                                                                       
                  Welcome!                    ▄▄█▀                  FishMol
                                          ▄▄███▀                 version 0.0.1
      ○                                ▄▄█████▀                          ○
           ○                        ▄▄████████▄                         /
                                ▄▄▄████████████▄                    ○--○
         ○                ▄▄▄████████████████████▄▄                     \ 
                    ▄▄▄██████▀███████████████████████▄▄                  ○--○           ▄
                ▄▄████████████ ██████████████████████████▄▄             /           ▄▄█▀
         ○   ▄█████████████████ ████████████████████████████▄▄         ○         ▄███▀
          ▄██████████▀▀▀████████ ██████████████████████████████▄▄             ▄█████▀
         ▄████████▀   ○  ▀███████ █████████████████████████████████▄▄      ▄▄█████▀
         ■▄███████▄     

In [3]:
from fishmol import trj, msd
from cage_data import cage1_info
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fishmol import style

In [16]:
from cage_data import cage1_info

waters = cage1_info.waters
water_indices = [[*water.values()] for water in waters]
print(water_indices)

[[14, 15, 16], [17, 18, 19], [143, 144, 145], [146, 147, 148], [272, 273, 274], [275, 276, 277], [401, 402, 403], [404, 405, 406]]


In [13]:
%%time
cell = cage1_info.cell
traj = trj.Trajectory(timestep = 5, data = "cage_data/cage1-500K.xyz", index = ":", cell = cell)

CPU times: user 45.8 s, sys: 2.61 s, total: 48.4 s
Wall time: 48.5 s


In [11]:
traj.cell

[[21.2944, 0.0, 0.0],
 [-4.6030371123, 20.7909480472, 0.0],
 [-0.9719093466, -1.2106211379, 15.1054299403]]

In [18]:
traj.frames[0][water_indices[0]].calc_com()

array([10.58631557, 19.90213351, 10.38000923])

In [37]:
from fishmol.utils import update_progress

water_coms = np.zeros((traj.nframes, 3*len(water_indices)))

for i, frame in enumerate(traj.frames):
    # Calculate com
    coms = [frame[water_idx].calc_com() for water_idx in water_indices]
    water_coms[i] = np.asarray(coms).flatten()
    update_progress(i/traj.nframes)
    
# Add data to dateframe
columns=[]
for i in range(1, len(water_indices) + 1):
    columns += [f"water{i}_x", f"water{i}_y", f"water{i}_z"]

water_com_df = pd.DataFrame(columns=columns, data = water_coms)

# Write data to excel file
water_com_df.to_excel("test/cage1-500K-water-com.xlsx")
update_progress(1)

Progress: [■■■■■■■■■■■■■■■■■■■■] 100.0%


In [36]:
water_com_df.head()

,water1_x,water1_y,water1_z,water2_x,water2_y,water2_z,water3_x,water3_y,water3_z,water4_x,...,water5_z,water6_x,water6_y,water6_z,water7_x,water7_y,water7_z,water8_x,water8_y,water8_z
0,10.586316,19.902134,10.380009,0.624155,10.923925,8.298079,-1.967392,4.261146,12.241481,4.693672,...,4.725172,15.090375,8.649502,6.883228,17.759366,15.321196,2.865725,11.019290,3.578953,0.703805
1,10.597616,19.945151,10.393126,0.617881,10.942585,8.312331,-1.961926,4.253910,12.205821,4.689873,...,4.745156,15.076014,8.640429,6.849210,17.749958,15.278667,2.889409,11.049146,3.587807,0.663123
2,10.610208,20.003821,10.408405,0.610859,10.967922,8.330438,-1.953047,4.243309,12.156421,4.687198,...,4.767198,15.058083,8.628816,6.804086,17.738720,15.225162,2.920655,11.086905,3.600664,0.608757
3,10.622718,20.062112,10.416761,0.604688,10.993416,8.349495,-1.945208,4.229969,12.106099,4.687512,...,4.786539,15.040753,8.617851,6.760128,17.728982,15.181476,2.948253,11.122864,3.614650,0.558271
4,10.636664,20.122749,10.416813,0.597829,11.020753,8.373153,-1.941190,4.211549,12.054352,4.691621,...,4.808036,15.020218,8.605213,6.714651,17.717884,15.151479,2.970079,11.155729,3.629731,0.511643
